In [1]:
# Import Packages
import os
import sys
import time


import matplotlib
import matplotlib.pyplot as plt
import json
from web3 import Web3
import requests
import pickle
import logging

%matplotlib inline

### Fetching pool data

In [9]:
# Infura Details
PROJECT_ID = "1b0fc3af1d194e8ea8db248ccf926085"


# Etherscan API Details
ETHERSCAN_API_KEY = "P4XI3P1XKMUFPGIKGDNEJ8RAV6TBE317MW"
ETH_MAINNET = "https://api.etherscan.io/"

In [10]:
# Web3 Handle
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/"+str(PROJECT_ID)))

In [4]:
# Fetch ABI from contract address
## Needs Contract Address and API Key as an input

def etherscan_abi_request(_contractAddress, _apiKey):
    '''
        Etherscan ABI Request Method
        Args::
            _contractAddress: Ethereum Address of the contract
            _apiKey: Etherscan API Key
        Returns::
            ethResJS: Etherscan ABI request result in JSON format
    '''
    ETHERSCAN_ABI_REQUEST = "https://api.etherscan.io/api?module=contract&action=getabi&address={0}&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_ABI_REQUEST)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS

In [5]:
# Fetch transactions history from contract address
## Needs Contract Address and API Key as an input

def etherscan_transaction_history(_contractAddress, _apiKey):
    '''
    
        Etherscan Transaction History Method
        Args::
            _contractAddress: Ethereum Address of the contract
            _apiKey: Etherscan API Key
        Returns::
            ethResJS: Etherscan ABI request result in JSON format
    '''
#     ETHERSCAN_ABI_REQUEST = ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&startblock=0&endblock=99999999&page=1&offset=100&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_TRANSACTION_HISTORY)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS

In [6]:
# Fetch transactions history from contract address
## Needs Contract Address and API Key as an input

def etherscan_erc20_transaction_history(_contractAddress, _apiKey):
    '''
    
        Etherscan ERC20 Token Transaction History Method
        Args::
            _contractAddress: Ethereum Address of the contract
            _apiKey: Etherscan API Key
        Returns::
            ethResJS: Etherscan ABI request result in JSON format
    '''
#     ETHERSCAN_ABI_REQUEST = ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=tokentx&address={0}&startblock=0&endblock=99999999&page=1&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=tokentx&address={0}&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_TRANSACTION_HISTORY)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS

In [7]:
# Fetch transactions details from transaction hash
## Needs transaction hash and API Key as an input

def etherscan_get_transaction_receipt(_transactionHash, _apiKey):
    '''
        Etherscan Get Transaction Receipt Method
        Args::
            _transactionHash: Transaction hash of the transaction
            _apiKey: Etherscan API Key
        Returns::
            etherResJS: Etherscan Transaction Receipt in JSON Format
    '''
    ETHERSCAN_TRANSACTION_RECEIPT = "https://api.etherscan.io/api?module=proxy&action=eth_getTransactionReceipt&txhash={0}&apikey={1}".format(_transactionHash, _apiKey)
#     ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_TRANSACTION_RECEIPT)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS
    
    

In [13]:
## The addresses are taken from:
## https://docs.aave.com/developers/v/2.0/deployed-contracts/deployed-contracts
aave_lending_protocol_address = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
aave_lending_protocol_implementation_address = "0xC6845a5C768BF8D7681249f8927877Efda425baf"

# This ABI is from Implementation Address
aave_abi = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":false,"internalType":"address","name":"user","type":"address"},{"indexed":true,"internalType":"address","name":"onBehalfOf","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"borrowRateMode","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"borrowRate","type":"uint256"},{"indexed":true,"internalType":"uint16","name":"referral","type":"uint16"}],"name":"Borrow","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":false,"internalType":"address","name":"user","type":"address"},{"indexed":true,"internalType":"address","name":"onBehalfOf","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":true,"internalType":"uint16","name":"referral","type":"uint16"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"target","type":"address"},{"indexed":true,"internalType":"address","name":"initiator","type":"address"},{"indexed":true,"internalType":"address","name":"asset","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"premium","type":"uint256"},{"indexed":false,"internalType":"uint16","name":"referralCode","type":"uint16"}],"name":"FlashLoan","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"collateralAsset","type":"address"},{"indexed":true,"internalType":"address","name":"debtAsset","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"},{"indexed":false,"internalType":"uint256","name":"debtToCover","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"liquidatedCollateralAmount","type":"uint256"},{"indexed":false,"internalType":"address","name":"liquidator","type":"address"},{"indexed":false,"internalType":"bool","name":"receiveAToken","type":"bool"}],"name":"LiquidationCall","type":"event"},{"anonymous":false,"inputs":[],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"}],"name":"RebalanceStableBorrowRate","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"},{"indexed":true,"internalType":"address","name":"repayer","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Repay","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":false,"internalType":"uint256","name":"liquidityRate","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"stableBorrowRate","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"variableBorrowRate","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"liquidityIndex","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"variableBorrowIndex","type":"uint256"}],"name":"ReserveDataUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"}],"name":"ReserveUsedAsCollateralDisabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"}],"name":"ReserveUsedAsCollateralEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"},{"indexed":false,"internalType":"uint256","name":"rateMode","type":"uint256"}],"name":"Swap","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpaused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"reserve","type":"address"},{"indexed":true,"internalType":"address","name":"user","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Withdraw","type":"event"},{"inputs":[],"name":"FLASHLOAN_PREMIUM_TOTAL","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"LENDINGPOOL_REVISION","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MAX_NUMBER_RESERVES","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MAX_STABLE_RATE_BORROW_SIZE_PERCENT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"interestRateMode","type":"uint256"},{"internalType":"uint16","name":"referralCode","type":"uint16"},{"internalType":"address","name":"onBehalfOf","type":"address"}],"name":"borrow","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"onBehalfOf","type":"address"},{"internalType":"uint16","name":"referralCode","type":"uint16"}],"name":"deposit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"balanceFromBefore","type":"uint256"},{"internalType":"uint256","name":"balanceToBefore","type":"uint256"}],"name":"finalizeTransfer","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"receiverAddress","type":"address"},{"internalType":"address[]","name":"assets","type":"address[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"uint256[]","name":"modes","type":"uint256[]"},{"internalType":"address","name":"onBehalfOf","type":"address"},{"internalType":"bytes","name":"params","type":"bytes"},{"internalType":"uint16","name":"referralCode","type":"uint16"}],"name":"flashLoan","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getAddressesProvider","outputs":[{"internalType":"contract ILendingPoolAddressesProvider","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getConfiguration","outputs":[{"components":[{"internalType":"uint256","name":"data","type":"uint256"}],"internalType":"struct DataTypes.ReserveConfigurationMap","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"components":[{"components":[{"internalType":"uint256","name":"data","type":"uint256"}],"internalType":"struct DataTypes.ReserveConfigurationMap","name":"configuration","type":"tuple"},{"internalType":"uint128","name":"liquidityIndex","type":"uint128"},{"internalType":"uint128","name":"variableBorrowIndex","type":"uint128"},{"internalType":"uint128","name":"currentLiquidityRate","type":"uint128"},{"internalType":"uint128","name":"currentVariableBorrowRate","type":"uint128"},{"internalType":"uint128","name":"currentStableBorrowRate","type":"uint128"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"},{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"},{"internalType":"address","name":"interestRateStrategyAddress","type":"address"},{"internalType":"uint8","name":"id","type":"uint8"}],"internalType":"struct DataTypes.ReserveData","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveNormalizedIncome","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveNormalizedVariableDebt","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getReservesList","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"user","type":"address"}],"name":"getUserAccountData","outputs":[{"internalType":"uint256","name":"totalCollateralETH","type":"uint256"},{"internalType":"uint256","name":"totalDebtETH","type":"uint256"},{"internalType":"uint256","name":"availableBorrowsETH","type":"uint256"},{"internalType":"uint256","name":"currentLiquidationThreshold","type":"uint256"},{"internalType":"uint256","name":"ltv","type":"uint256"},{"internalType":"uint256","name":"healthFactor","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"user","type":"address"}],"name":"getUserConfiguration","outputs":[{"components":[{"internalType":"uint256","name":"data","type":"uint256"}],"internalType":"struct DataTypes.UserConfigurationMap","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtAddress","type":"address"},{"internalType":"address","name":"variableDebtAddress","type":"address"},{"internalType":"address","name":"interestRateStrategyAddress","type":"address"}],"name":"initReserve","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract ILendingPoolAddressesProvider","name":"provider","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"collateralAsset","type":"address"},{"internalType":"address","name":"debtAsset","type":"address"},{"internalType":"address","name":"user","type":"address"},{"internalType":"uint256","name":"debtToCover","type":"uint256"},{"internalType":"bool","name":"receiveAToken","type":"bool"}],"name":"liquidationCall","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"address","name":"user","type":"address"}],"name":"rebalanceStableBorrowRate","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"rateMode","type":"uint256"},{"internalType":"address","name":"onBehalfOf","type":"address"}],"name":"repay","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"configuration","type":"uint256"}],"name":"setConfiguration","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"val","type":"bool"}],"name":"setPause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"address","name":"rateStrategyAddress","type":"address"}],"name":"setReserveInterestRateStrategyAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"bool","name":"useAsCollateral","type":"bool"}],"name":"setUserUseReserveAsCollateral","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"rateMode","type":"uint256"}],"name":"swapBorrowRateMode","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"to","type":"address"}],"name":"withdraw","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"}]'

In [21]:
erc_20_abi = '[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_upgradedAddress","type":"address"}],"name":"deprecate","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"deprecated","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_evilUser","type":"address"}],"name":"addBlackList","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"upgradedAddress","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"balances","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"maximumFee","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"_totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"unpause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_maker","type":"address"}],"name":"getBlackListStatus","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"},{"name":"","type":"address"}],"name":"allowed","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"paused","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"who","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"pause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getOwner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"newBasisPoints","type":"uint256"},{"name":"newMaxFee","type":"uint256"}],"name":"setParams","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"issue","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"redeem","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"remaining","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"basisPointsRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"isBlackListed","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_clearedUser","type":"address"}],"name":"removeBlackList","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"MAX_UINT","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_blackListedUser","type":"address"}],"name":"destroyBlackFunds","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[{"name":"_initialSupply","type":"uint256"},{"name":"_name","type":"string"},{"name":"_symbol","type":"string"},{"name":"_decimals","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"amount","type":"uint256"}],"name":"Issue","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"amount","type":"uint256"}],"name":"Redeem","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"newAddress","type":"address"}],"name":"Deprecate","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"feeBasisPoints","type":"uint256"},{"indexed":false,"name":"maxFee","type":"uint256"}],"name":"Params","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_blackListedUser","type":"address"},{"indexed":false,"name":"_balance","type":"uint256"}],"name":"DestroyedBlackFunds","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_user","type":"address"}],"name":"AddedBlackList","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_user","type":"address"}],"name":"RemovedBlackList","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[],"name":"Pause","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpause","type":"event"}]'

In [14]:
aave_contract = w3.eth.contract(
address=w3.toChecksumAddress(aave_lending_protocol_address),
abi=aave_abi
)

In [17]:
# Getting list of assets from Aave Contract
assets_list = aave_contract.functions.getReservesList().call()

In [18]:
# List of assets
print(assets_list)

['0xdAC17F958D2ee523a2206206994597C13D831ec7', '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e', '0xE41d2489571d322189246DaFA5ebDe1F4699F498', '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9', '0x0D8775F648430679A709E98d2b0Cb6250d2887EF', '0x4Fabb145d64652a948d72533023f6E7A623C7C53', '0x6B175474E89094C44Da98b954EedeAC495271d0F', '0xF629cBd94d3791C9250152BD8dfBDF380E2a3B9c', '0xdd974D5C2e2928deA5F71b9825b8b646686BD200', '0x514910771AF9Ca656af840dff83E8264EcF986CA', '0x0F5D2fB29fb7d3CFeE444a200298f468908cC942', '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0x408e41876cCCDC0F92210600ef50372656052a38', '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', '0x57Ab1ec28D129707052df4dF418D58a2D46d5f51', '0x0000000000085d4780B73119b644AE5ecd22b376', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', '0xD533a949740bb3306d119CC777fa900bA034cd52', '0x056Fd409E1d7A124BD7017459dFEa2

In [19]:
# First Asset
print(assets_list[0])

0xdAC17F958D2ee523a2206206994597C13D831ec7


In [ ]:
## Aave Rserve Data Variables
## Located at: https://github.com/aave/aave-protocol/blob/master/contracts/lendingpool/LendingPoolDataProvider.sol
# function getReserveData(address _reserve)
# external
# view
# returns (
#     uint256 totalLiquidity,
#     uint256 availableLiquidity,
#     uint256 totalBorrowsStable,
#     uint256 totalBorrowsVariable,
#     uint256 liquidityRate,
#     uint256 variableBorrowRate,
#     uint256 stableBorrowRate,
#     uint256 averageStableBorrowRate,
#     uint256 utilizationRate,
#     uint256 liquidityIndex,
#     uint256 variableBorrowIndex,
#     address aTokenAddress,
#     uint40 lastUpdateTimestamp
# )

In [43]:
# Asset symbols and reserves 
aave_assets_list = {}
# Asset symbols and addresses
aave_assets_address = {}
# Looping through the assets and getting their symbols and reserves
counter = 0
for currentAsset in assets_list:
    counter += 1
    try:
        print("Current Asset Address: "+str(counter))
        
        print(currentAsset)
        asset_contract = w3.eth.contract(
        address=w3.toChecksumAddress(currentAsset),
        abi=erc_20_abi
        )
        symbol = asset_contract.functions.symbol().call()
        decimals = asset_contract.functions.decimals().call()
        print(symbol)
        print(decimals)
        print()
        
    except Exception as e:
        print("Exception Occured")
        print(e)

Current Asset Address: 1
0xdAC17F958D2ee523a2206206994597C13D831ec7
USDT
6

Current Asset Address: 2
0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599
WBTC
8

Current Asset Address: 3
0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
WETH
18

Current Asset Address: 4
0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e
YFI
18

Current Asset Address: 5
0xE41d2489571d322189246DaFA5ebDe1F4699F498
ZRX
18

Current Asset Address: 6
0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984
UNI
18

Current Asset Address: 7
0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9
AAVE
18

Current Asset Address: 8
0x0D8775F648430679A709E98d2b0Cb6250d2887EF
BAT
18

Current Asset Address: 9
0x4Fabb145d64652a948d72533023f6E7A623C7C53
BUSD
18

Current Asset Address: 10
0x6B175474E89094C44Da98b954EedeAC495271d0F
DAI
18

Current Asset Address: 11
0xF629cBd94d3791C9250152BD8dfBDF380E2a3B9c
ENJ
18

Current Asset Address: 12
0xdd974D5C2e2928deA5F71b9825b8b646686BD200
KNC
18

Current Asset Address: 13
0x514910771AF9Ca656af840dff83E8264EcF986CA
LINK
18

Curr

In [32]:
assets_reserves_list = aave_contract.functions.getReserveData(assets_list[0]).call()

In [33]:
assets_reserves_list

((18447682511281904943104,),
 1079648877148964595514283880,
 1120411989987272995935928784,
 20189229043505455811417447,
 31023423026036504497323838,
 115511711513018252248661919,
 1644583717,
 '0x3Ed3B47Dd13EC9a98b44e6204A523E766B225811',
 '0xe91D55AB2240594855aBd11b3faAE801Fd4c4687',
 '0x531842cEbbdD378f8ee36D171d6cC9C4fcf475Ec',
 '0x515E87cb3fec986050F202a2bbfa362A2188bc3F',
 0)

In [ ]:
# returns (
#     uint256 totalLiquidity,
#     uint256 availableLiquidity,
#     uint256 totalBorrowsStable,
#     uint256 totalBorrowsVariable,
#     uint256 liquidityRate,
#     uint256 variableBorrowRate,
#     uint256 stableBorrowRate,
#     uint256 averageStableBorrowRate,
#     uint256 utilizationRate,
#     uint256 liquidityIndex,
#     uint256 variableBorrowIndex,
#     address aTokenAddress,
#     uint40 lastUpdateTimestamp
# )

In [46]:
assets_reserves_list[1]/pow(10, 27)

1.0796488771489645

In [ ]:
1057684701